# Notebook 02 — Correlation Filtering & Cointegration Testing
### Adaptive Statistical Arbitrage — Ayush Arora (MQMS2404)

In this notebook, we:
- Load cleaned NIFTY 100 price data
- Compute daily returns
- Filter stock pairs using correlation threshold
- Perform Engle–Granger Cointegration Test
- Identify statistically significant cointegrated pairs

These pairs will be used for spread construction & mean reversion modeling.

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint

# Load cleaned dataset from Notebook 01
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True)
prices.head()

,ABB.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,ASHOKLEY.NS,ASIANPAINT.NS,AUROPHARMA.NS,AXISBANK.NS,BAJAJ-AUTO.NS,...,TATASTEEL.NS,TECHM.NS,TITAN.NS,TORNTPHARM.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,VEDL.NS,WIPRO.NS,YESBANK.NS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,1131.143555,70.761375,301.772858,195.772980,1084.179321,21.544607,684.931519,532.220825,486.962860,1845.359375,...,28.898026,462.629150,362.723755,494.190887,259.655640,808.554260,2548.028809,70.610985,93.508194,147.012482
2015-01-02,1119.410767,71.107956,301.584015,198.523727,1086.056885,22.124655,708.611328,534.770935,497.853149,1845.058228,...,29.348158,464.893738,365.588867,514.467651,250.206848,809.523132,2624.206055,71.593269,94.337662,150.751801
2015-01-05,1119.671387,72.284897,305.786469,198.265823,1089.667847,23.947659,708.565735,534.510925,500.999237,1851.793579,...,29.773287,457.137268,368.549530,512.181091,253.434357,806.907471,2629.565430,70.788139,94.506920,151.075287
2015-01-06,1109.503052,71.736137,303.944946,190.959259,1057.216553,23.429758,691.651428,513.663147,483.090790,1837.344727,...,28.329998,452.233887,355.943054,488.625641,250.768082,806.471497,2555.780518,67.326035,92.297859,146.841217
2015-01-07,1093.511230,71.100731,303.236694,189.197098,1065.498169,24.714149,705.548584,521.808899,482.703583,1841.634277,...,27.786966,450.120880,357.757538,486.015411,260.357269,869.103882,2545.683838,67.309937,91.595390,144.120010


## Step 1: Compute Daily Returns
Daily returns capture co-movement strength among stocks.

In [2]:
returns = prices.pct_change(fill_method=None).dropna()
returns.head()

,ABB.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,ASHOKLEY.NS,ASIANPAINT.NS,AUROPHARMA.NS,AXISBANK.NS,BAJAJ-AUTO.NS,...,TATASTEEL.NS,TECHM.NS,TITAN.NS,TORNTPHARM.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,VEDL.NS,WIPRO.NS,YESBANK.NS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,-0.010373,0.004898,-0.000626,0.014051,0.001732,0.026923,0.034573,0.004791,0.022364,-0.000163,...,0.015577,0.004895,0.007899,0.041030,-0.036390,0.001198,0.029897,0.013911,0.008871,0.025435
2015-01-05,0.000233,0.016551,0.013935,-0.001299,0.003325,0.082397,-0.000064,-0.000486,0.006319,0.003650,...,0.014486,-0.016684,0.008098,-0.004445,0.012899,-0.003231,0.002042,-0.011246,0.001794,0.002146
2015-01-06,-0.009082,-0.007592,-0.006022,-0.036852,-0.029781,-0.021626,-0.023871,-0.039003,-0.035745,-0.007803,...,-0.048476,-0.010726,-0.034206,-0.045990,-0.010521,-0.000540,-0.028060,-0.048908,-0.023375,-0.028026
2015-01-07,-0.014413,-0.008858,-0.002330,-0.009228,0.007833,0.054819,0.020093,0.015858,-0.000802,0.002335,...,-0.019168,-0.004672,0.005098,-0.005342,0.038239,0.077662,-0.003951,-0.000239,-0.007611,-0.018532
2015-01-08,0.023685,0.039403,0.041887,0.028623,0.017939,0.026823,0.063043,0.019591,0.006618,0.001308,...,0.015557,0.016736,-0.004538,0.018597,0.064319,-0.004180,0.025338,0.007177,0.007207,0.015581


## Step 2: Compute Correlation Matrix
We use absolute correlations to filter out unrelated stock pairs.

In [3]:
corr = returns.corr().abs()
corr

,ABB.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,ASHOKLEY.NS,ASIANPAINT.NS,AUROPHARMA.NS,AXISBANK.NS,BAJAJ-AUTO.NS,...,TATASTEEL.NS,TECHM.NS,TITAN.NS,TORNTPHARM.NS,TVSMOTOR.NS,UBL.NS,ULTRACEMCO.NS,VEDL.NS,WIPRO.NS,YESBANK.NS
ABB.NS,1.000000,0.240846,0.284810,0.284951,0.146527,0.249246,0.196791,0.145733,0.230399,0.217666,...,0.231241,0.154693,0.182341,0.147543,0.235523,0.189656,0.259018,0.210808,0.156433,0.092524
ADANIENT.NS,0.240846,1.000000,0.577809,0.431101,0.202594,0.306013,0.232409,0.236611,0.289828,0.228060,...,0.339162,0.200784,0.242511,0.172387,0.251664,0.240276,0.331736,0.319890,0.192042,0.178274
ADANIPORTS.NS,0.284810,0.577809,1.000000,0.481206,0.224962,0.337253,0.282517,0.267177,0.353778,0.290815,...,0.385643,0.254252,0.284413,0.176609,0.311156,0.262534,0.390931,0.357166,0.222230,0.195766
AMBUJACEM.NS,0.284951,0.431101,0.481206,1.000000,0.238421,0.372958,0.322435,0.252983,0.356599,0.307610,...,0.374169,0.206060,0.292738,0.174849,0.313284,0.274080,0.572583,0.328327,0.223736,0.160692
APOLLOHOSP.NS,0.146527,0.202594,0.224962,0.238421,1.000000,0.215456,0.236704,0.229402,0.229977,0.222623,...,0.234909,0.227768,0.209256,0.227592,0.196127,0.223073,0.285604,0.216307,0.221713,0.096786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UBL.NS,0.189656,0.240276,0.262534,0.274080,0.223073,0.285288,0.266549,0.243219,0.253848,0.244951,...,0.257787,0.176152,0.253614,0.171316,0.246271,1.000000,0.318858,0.239958,0.160663,0.110601
ULTRACEMCO.NS,0.259018,0.331736,0.390931,0.572583,0.285604,0.386396,0.417826,0.279823,0.419652,0.341815,...,0.430534,0.271030,0.319111,0.183937,0.359339,0.318858,1.000000,0.350066,0.277840,0.153781
VEDL.NS,0.210808,0.319890,0.357166,0.328327,0.216307,0.341789,0.187592,0.293303,0.343989,0.276398,...,0.628697,0.216191,0.262948,0.190751,0.282340,0.239958,0.350066,1.000000,0.227273,0.170663
WIPRO.NS,0.156433,0.192042,0.222230,0.223736,0.221713,0.203744,0.204841,0.223845,0.228124,0.220209,...,0.259795,0.480654,0.208515,0.170287,0.172201,0.160663,0.277840,0.227273,1.000000,0.033485


## Step 3: Select Candidate Pairs Using Correlation Threshold
We only test pairs with correlation > 0.3 to reduce computation.

In [4]:
pairs = []
threshold = 0.3  # correlation filter

for a in prices.columns:
    for b in prices.columns:
        if a < b and corr.loc[a, b] > threshold:
            pairs.append((a, b))

len(pairs), pairs[:10]

(1024,
 [('ABB.NS', 'LT.NS'),
  ('ABB.NS', 'SIEMENS.NS'),
  ('ADANIENT.NS', 'ADANIPORTS.NS'),
  ('ADANIENT.NS', 'AMBUJACEM.NS'),
  ('ADANIENT.NS', 'ASHOKLEY.NS'),
  ('ADANIENT.NS', 'BAJAJFINSV.NS'),
  ('ADANIENT.NS', 'BAJFINANCE.NS'),
  ('ADANIENT.NS', 'BANKBARODA.NS'),
  ('ADANIENT.NS', 'BHARATFORG.NS'),
  ('ADANIENT.NS', 'BHEL.NS')])

## Step 4: Perform Engle–Granger Cointegration Test
This identifies pairs with long-term equilibrium relationships.

In [5]:
results = []

for a, b in pairs:
    series_a = prices[a].dropna()
    series_b = prices[b].dropna()
    score, pvalue, _ = coint(series_a, series_b)
    results.append([a, b, pvalue])

df_results = pd.DataFrame(results, columns=["Stock A", "Stock B", "p-value"])
df_results = df_results.sort_values("p-value")
df_results.head(20)

,Stock A,Stock B,p-value
578,DABUR.NS,HINDUNILVR.NS,0.000100
529,CHOLAFIN.NS,JINDALSTEL.NS,0.002127
243,BAJAJ-AUTO.NS,SIEMENS.NS,0.002271
124,ASHOKLEY.NS,EICHERMOT.NS,0.002356
670,HAVELLS.NS,JSWSTEEL.NS,0.002457
401,BHARATFORG.NS,TVSMOTOR.NS,0.003178
100,AMBUJACEM.NS,SBIN.NS,0.003719
981,SRF.NS,TATACHEM.NS,0.003807
838,JINDALSTEL.NS,ULTRACEMCO.NS,0.003829
68,ADANIPORTS.NS,ULTRACEMCO.NS,0.004118


## Step 5: Save Cointegration Results

In [6]:
df_results.to_csv("data/cointegration_results.csv", index=False)
print("Cointegration results saved to data/cointegration_results.csv")

Cointegration results saved to data/cointegration_results.csv


### Summary
- Loaded cleaned NIFTY 100 price data
- Computed correlations
- Filtered pairs with correlation > 0.3
- Performed Engle–Granger cointegration test
- Saved sorted results for use in Notebook 03

Next: **Notebook 03 — Spread Construction (OLS Regression + ADF Test)**